In [1]:
!pip install transformers datasets torch seqeval pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 s

In [2]:
from google.colab import files
uploaded = files.upload()

Saving amharic_ner.conll to amharic_ner.conll


In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Define your label list based on your annotation schema
label_list = [
    "O",         # Outside of any entity
    "B-PRODUCT", # Beginning of product
    "I-PRODUCT", # Inside product
    "B-PRICE",   # Beginning of price
    "I-PRICE",   # Inside price
    "B-LOC",     # Beginning of location
    "I-LOC"      # Inside location
]

# Function to parse the uploaded CoNLL file
def parse_conll(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    sentences = []
    current_sentence = []

    for line in lines:
        line = line.strip()
        if not line:
            if current_sentence:
                sentences.append(current_sentence)
                current_sentence = []
        else:
            parts = line.split()
            token = parts[0]
            label = parts[-1] if len(parts) > 1 else "O"
            current_sentence.append((token, label))

    if current_sentence:
        sentences.append(current_sentence)

    return sentences

# Parse the uploaded file
sentences = parse_conll("amharic_ner.conll")

# Convert to pandas DataFrame for easier handling
data = []
for sentence in sentences:
    tokens = [token for token, label in sentence]
    labels = [label for token, label in sentence]
    data.append({'tokens': tokens, 'ner_tags': labels})

df = pd.DataFrame(data)

# Split into train and validation sets (80/20 split)
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

# Create label mappings
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}

print(f"Number of training examples: {len(dataset['train'])}")
print(f"Number of validation examples: {len(dataset['validation'])}")
print("\nSample from training set:")
print(dataset['train'][0])

Number of training examples: 4
Number of validation examples: 1

Sample from training set:
{'tokens': ['💥💥...................................💥💥', '📌1', 'pairs', 'Sneaker', 'Crease', 'Protector', 'ዋጋ፦', '💵🏷', '400', 'ብር', '♦️ውስን', 'ፍሬ', 'ነው', 'ያለው🔥🔥🔥', '🏢', 'አድራሻ👉', '📍♦️#መገናኛ_መሰረት_ደፋር_ሞል_ሁለተኛ_ፎቅ', 'ቢሮ', 'ቁ.', 'S05/S06', '💧💧💧💧', '📲', '0902660722', '📲', '0928460606', '🔖', '💬በTelegram', 'ለማዘዝ', '⤵️', 'ይጠቀሙ🔽', '@zemencallcenter', '@zemenexpressadmin', 'ለተጨማሪ', 'ማብራሪያ', 'የቴሌግራም', 'ገፃችን⤵️', 'https://telegram.me/zemenexpress', '💥💥...................................💥💥'], 'ner_tags': ['O', 'O', 'B-PRODUCT', 'B-PRODUCT', 'O', 'I-PRODUCT', 'B-PRICE', 'O', 'I-PRICE', 'I-PRICE', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'O', 'I-LOC', 'O', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], '__index_level_0__': 4}


In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "xlm-roberta-base"  # Using XLM-Roberta as it's mentioned in the task

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Special tokens for Amharic (if needed)
# tokenizer.add_tokens(["[AMH]", "[/AMH]"])

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

# Resize model embeddings if we added new tokens
# model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])  # Assign label to first subword
            else:
                label_ids.append(-100)  # Ignore subsequent subwords
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

sample = tokenized_datasets["validation"][0]
print("Tokens:", tokenizer.convert_ids_to_tokens(sample["input_ids"]))
print("Labels:", sample["labels"])

Tokens: ['<s>', '▁', '💥', '💥', '...', '................', '................', '💥', '💥', '▁3', 'pc', 's', '▁Bo', 'ttle', '▁Stop', 'per', '▁', '👍', 'በ', 'ማንኛውም', '▁', 'ጠር', 'ሙስ', '▁', 'ጫ', 'ፍ', '▁የሚ', 'ገ', 'ጠ', 'ም', '▁', '👍', 'ለ', 'ዘ', 'ይት', '▁እና', '▁መ', 'ሰል', '▁ነገሮች', '▁መቀ', 'ነ', 'ሻ', '▁የሚሆን', '▁', '👍', 'በ', 'አ', 'ግባ', 'ቡ', '▁እየ', 'መ', 'ጠን', 'ን', '▁ለመ', 'ጠቀም', '▁ተ', 'መራ', 'ጭ', '▁ዋጋ', '፦', '▁', '💲', '<unk>', '▁3', 'ፍ', 'ሬ', '➡', '️', '▁400', '▁ብር', '▁♦', '️', 'ውስ', 'ን', '▁ፍ', 'ሬ', '▁ነው', '▁ያለው', '▁', '🔥', '🔥', '🔥', '▁', '🏢', '▁አድራሻ', '👉', '▁', '📍', '♦', '️', '#', 'መ', 'ገና', 'ኛ', '_', 'መሰ', 'ረት', '_', 'ደ', 'ፋር', '_', 'ሞ', 'ል', '_', 'ሁለተኛ', '_', 'ፎ', 'ቅ', '▁ቢሮ', '▁ቁ', '.', '▁S', '05', '/', 'S', '06', '▁', '💧', '💧', '💧', '💧', '▁', '📲', '▁09', '02', '660', '722', '▁', '📲', '▁09', '28', '460', '606', '▁', '🔖', '▁', '💬', 'በ', 'Tele', 'gram', '▁ለማ', 'ዘ', 'ዝ', '▁', '⤵', '️', '▁ይ', 'ጠቀሙ', '<unk>', '▁@', 'zem', 'en', 'call', 'center', '▁@', 'zem', 'en', 'express', 'admin', '▁ለተ', 'ጨ', 'ማሪ', '▁ማብራሪ

In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./amharic_ner_results",
    eval_strategy="epoch",  # Changed from evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir='./logs',
    logging_steps=50,
    report_to="none"  # Change to "wandb" if you want to use Weights & Biases
)

In [29]:
from transformers import Trainer, DataCollatorForTokenClassification
import numpy as np
from datasets import load_metric

# Define data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Define metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (where label is -100)
    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

/tmp/ipython-input-29-300219259.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.883756,0.000000,0.000000,0.000000,0.807692
2,No log,0.756806,0.000000,0.000000,0.000000,0.807692
3,No log,0.697543,0.000000,0.000000,0.000000,0.807692


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=3, training_loss=1.1117383639017742, metrics={'train_runtime': 192.8671, 'train_samples_per_second': 0.062, 'train_steps_per_second': 0.016, 'total_flos': 1175888558592.0, 'train_loss': 1.1117383639017742, 'epoch': 3.0})


Evaluation results:
Precision: 0.133
Recall: 0.400
F1 Score: 0.200
Accuracy: 0.365


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
# Save the model and tokenizer
save_path = "./amharic_ner_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model saved to {save_path}")

# Optional: Save to Google Drive for long-term storage
from google.colab import drive
drive.mount('/content/drive')  # Follow the authorization prompt
!cp -r {save_path} "/content/drive/MyDrive/"
print("Model also saved to Google Drive")

Model saved to ./amharic_ner_model
Mounted at /content/drive
Model also saved to Google Drive


In [22]:
# Import torch at the top of your notebook (if not already present)
import torch

# Load the saved model
ner_pipeline = pipeline(
    "ner",
    model=save_path,
    tokenizer=save_path,
    device=0 if torch.cuda.is_available() else -1,  # Now torch is defined
    aggregation_strategy="simple"  # Groups tokens into full entities
)

# Sample Amharic text (replace with your own)
sample_text = "አዲስ አበባ ውስጥ ለልጅ እጅ ስራ ቤት እቃ 2500 ብር ይሸጣል። ደረሰኝ አገልግሎት ነፃ ነው።"

# Run prediction
results = ner_pipeline(sample_text)

# Print results
print("\nEntity Extraction Results:")
for entity in results:
    print(f"Entity: {entity['word']} | Type: {entity['entity_group']} | Confidence: {entity['score']:.2f}")

Device set to use cuda:0



Entity Extraction Results:
Entity: አዲስ አበባ ውስጥ ለልጅ እጅ ስራ ቤት እቃ 2500 ብር ይሸጣል። ደረሰኝ አገልግሎት ነፃ ነው። | Type: PRICE | Confidence: 0.25


In [30]:
# Run this cell to rename the notebook
from google.colab import drive
drive.mount('/content/drive')
!mv '/content/your_notebook.ipynb' '/content/amharic_ner_task3.ipynb'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mv: cannot stat '/content/your_notebook.ipynb': No such file or directory


In [31]:
!apt-get install git -qq  # Install Git

In [32]:
!git config --global user.email "carmel.zerihun@gmail.com"

In [33]:
!git config --global user.name "CarmelZe"

In [34]:
!git clone https://github.com/CarmelZe/Building-an-Amharic-E-commerce-Data-Extractor.git

Cloning into 'Building-an-Amharic-E-commerce-Data-Extractor'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 114 (delta 4), reused 108 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (114/114), 7.51 MiB | 14.85 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [38]:
%cd Building-an-Amharic-E-commerce-Data-Extractor

/content/Building-an-Amharic-E-commerce-Data-Extractor


In [39]:
!git checkout -b task-3  # Create and switch to the task-3 branch

Switched to a new branch 'task-3'


In [43]:
!git add amharic_ner_task3.ipynb  # Stage the notebook

fatal: pathspec 'amharic_ner_task3.ipynb' did not match any files


In [44]:
!git add .

In [45]:
!git commit -m "Add Task 3 notebook: Amharic NER fine-tuning"

On branch task-3
nothing to commit, working tree clean


In [46]:
!git push origin task-3

fatal: could not read Username for 'https://github.com': No such device or address
